In [1]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import binarize
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

import numpy as np
import scipy as sp
import scipy.io as sio

import h5py

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from deepsea import *

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5005)


In [76]:
train_X, train_Y = loaddata("data/deepsea/", "train")

In [75]:
test_X, test_Y = loaddata("data/deepsea/", "test")

In [74]:
valid_X, valid_Y = loaddata("data/deepsea/", "valid")

# Preparing training data for deeperSEA

Extracting common histone mark signals (marks below) in 6 celltypes.   
Also extracting "CTCF"  signal in the 6 celltypes to use a prediction target.

**Marks**: 'H3K4me3', 'H3K4me2', 'H3K4me1', 'H2AZ', 'H3K36me3', 'H4K20me1',
       'H3K27ac', 'H3K27me3', 'H3K9me3', 'H3K79me2', 'H3K9ac'  
**Celltypes**: 'H1-hESC', 'NH-A', 'NHEK', 'K562', 'NHDF-Ad', 'NHLF'

In [124]:
celltypes

['H1-hESC', 'NH-A', 'NHEK', 'K562', 'NHDF-Ad', 'NHLF']

In [53]:
raw = open("data/deepsea/features.txt").readlines()[1:-1]
indexmap = dict([(tuple(raw[i].split("\t")[1:3]), i) for i in range(len(raw))])

In [88]:
marks = np.load("data/deepersea/marks.npy")

In [49]:
celltypes = np.load("data/deepersea/celltypes.npy")

In [73]:
targets = np.load("data/deepersea/targets.npy")

In [131]:
data = test_Y
tf = "CTCF"

X = None
Y = None
for c in celltypes:
    indexes = [indexmap[(c,m)] for m in marks]
    if not X is None:
        X = np.concatenate([X, data[:, indexes]], axis=0)
        Y = np.concatenate([Y, data[:, indexmap[(c, tf)]]], axis=0)
    else:
        X = data[:, indexes]
        Y = data[:, indexmap[(c, tf)]]
print X.shape
print Y.shape

(2730144, 11)
(2730144,)


In [132]:
np.save("data/deepersea/testx", X)

In [133]:
np.save("data/deepersea/testy", Y)